### Creating a logistic regression to classify Lyon vs. Monaco

- 6th places vs 7th


In [1]:
import numpy as np
import pandas as pd
from statsbombpy import sb
pd.set_option('display.max_columns', None)
from mplsoccer import Pitch, Sbopen
from mplsoccer import VerticalPitch,Pitch
import networkx as nx
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import networkx as nx
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import math 


c:\Users\Barre\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\Barre\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
c:\Users\Barre\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\.libs\libopenblas.fb5ae2tyxyh2ijrdkgdgq3xbklktf43h.gfortran-win_amd64.dll
c:\Users\Barre\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
def digraph(data):
        G = nx.DiGraph()
        pass_counts = {}
        players = data["player_name"].unique()
        G.add_nodes_from(players)
        for _, row in data.iterrows():
                key = (row["player_name"], row["pass_recipient_name"])
                if key in pass_counts:
                        pass_counts[key] += 1
                else:
                        pass_counts[key] = 1

        for key, weight in pass_counts.items():
                G.add_edge(key[0], key[1], weight=weight)
        
        return G

In [3]:
parser = Sbopen()
df_competitions = parser.competition()
df_competitions

,competition_id,season_id,country_name,competition_name,competition_gender,competition_youth,competition_international,season_name,match_updated,match_updated_360,match_available_360,match_available
0,9,27,Germany,1. Bundesliga,male,False,False,2015/2016,2023-08-17T23:51:11.837478,None,None,2023-08-17T23:51:11.837478
1,16,4,Europe,Champions League,male,False,False,2018/2019,2023-03-07T12:20:48.118250,2021-06-13T16:17:31.694,None,2023-03-07T12:20:48.118250
2,16,1,Europe,Champions League,male,False,False,2017/2018,2021-08-27T11:26:39.802832,2021-06-13T16:17:31.694,None,2021-01-23T21:55:30.425330
3,16,2,Europe,Champions League,male,False,False,2016/2017,2021-08-27T11:26:39.802832,2021-06-13T16:17:31.694,None,2020-07-29T05:00
4,16,27,Europe,Champions League,male,False,False,2015/2016,2021-08-27T11:26:39.802832,2021-06-13T16:17:31.694,None,2020-07-29T05:00
...,...,...,...,...,...,...,...,...,...,...,...,...
62,55,43,Europe,UEFA Euro,male,False,True,2020,2023-02-24T21:26:47.128979,2023-04-27T22:38:34.970148,2023-04-27T22:38:34.970148,2023-02-24T21:26:47.128979
63,35,75,Europe,UEFA Europa League,male,False,False,1988/1989,2023-06-18T19:28:39.443883,2021-06-13T16:17:31.694,None,2023-06-18T19:28:39.443883
64,53,106,Europe,UEFA Women's Euro,female,False,True,2022,2023-07-17T21:19:03.032991,2023-07-17T21:21:56.497106,2023-07-17T21:21:56.497106,2023-07-17T21:19:03.032991
65,72,107,International,Women's World Cup,female,False,True,2023,2023-09-01T12:34:19.705316,2023-09-01T12:35:45.762196,2023-09-01T12:35:45.762196,2023-09-01T12:34:19.705316


In [4]:
teams = parser.match(competition_id=7, season_id=27)


##  Pseudo code
make data frame
Y- team with X1 = between centrality ...
- loop through df_match for each match
- if statement for same game just remove
    
- if away_team_name == Bayern Munich or Borussia Dortmund
    append team name
- make digraph

- compute centrality measures


In [6]:
def pass_data_monaco(match_id):
    df, related, freeze, tactics = parser.event(match_id)
    sub = df.loc[df["type_name"] == "Substitution"].loc[df["team_name"] == "AS Monaco"].iloc[0]["index"]

    mask_monaco = (df.type_name == 'Pass') & (df.team_name == "AS Monaco") & (df.index < sub) & (df.outcome_name.isnull()) & (df.sub_type_name != "Throw-in")
    df_pass = df.loc[mask_monaco, ['x', 'y', 'end_x', 'end_y', "player_name", "pass_recipient_name"]]
    df_pass["player_name"] = df_pass["player_name"].apply(lambda x: str(x).split()[-1])
    df_pass["pass_recipient_name"] = df_pass["pass_recipient_name"].apply(lambda x: str(x).split()[-1])
    return df_pass  



In [7]:
def pass_data_lyon(match_id):
    df, related, freeze, tactics = parser.event(match_id)

    sub = df.loc[df["type_name"] == "Substitution"].loc[df["team_name"] == "Lyon"].iloc[0]["index"]
    mask_lyon = (df.type_name == 'Pass') & (df.team_name == "Lyon") & (df.index < sub) & (df.outcome_name.isnull()) & (df.sub_type_name != "Throw-in")
    df_pass = df.loc[mask_lyon, ['x', 'y', 'end_x', 'end_y', "player_name", "pass_recipient_name"]]
    df_pass["player_name"] = df_pass["player_name"].apply(lambda x: str(x).split()[-1])
    df_pass["pass_recipient_name"] = df_pass["pass_recipient_name"].apply(lambda x: str(x).split()[-1])
    return df_pass

In [9]:
def avg(dictionary):
    total = sum(dictionary.values())

    average = total / len(dictionary)
    
    return average

In [1]:


def get_data(dat):
    data = pd.DataFrame()
    for index, row in dat.iterrows():
        if (row['home_team_name'] == 'AS Monaco'):
            match_id = row['match_id']
            passing_data_monaco = pass_data_monaco(match_id)
            
            G = digraph(passing_data_monaco)
            betweenness_centrality = avg(nx.betweenness_centrality(G))
            eigenvector_centrality = max(nx.eigenvector_centrality(G).values())
            katz_centrality = max(nx.katz_centrality(G).values())
            page_rank = max(nx.pagerank(G).values())
            
            match_data = {
                'Match ID': match_id,
                'Team': 'AS Monaco',
                'Betweenness Centrality': betweenness_centrality,
                'Eigenvector Centrality': eigenvector_centrality,
                'Katz Centrality': katz_centrality,
                'Page Rank': page_rank,
                'Average Clustering': (nx.average_clustering(G))
            }
            data = data.append(match_data, ignore_index=True)
        if (row['away_team_name'] == 'AS Monaco'):
            match_id = row['match_id']
            passing_data_monaco = pass_data_monaco(match_id)
            G = digraph(passing_data_monaco)
            betweenness_centrality = avg(nx.betweenness_centrality(G))
            eigenvector_centrality = max(nx.eigenvector_centrality(G).values())
            katz_centrality = max(nx.katz_centrality(G).values())
            page_rank = max(nx.pagerank(G).values())
        
            match_data = {
                'Match ID': match_id,
                'Team': 'AS Monaco',
                'Betweenness Centrality': betweenness_centrality,
                'Eigenvector Centrality': eigenvector_centrality,
                'Katz Centrality': katz_centrality,
                'Page Rank': page_rank,
                'Average Clustering': (nx.average_clustering(G))
            }
            data = data.append(match_data, ignore_index=True)

        if (row['home_team_name'] == 'Lyon'):
            match_id = row['match_id']
            passing_data_lyon = pass_data_lyon(match_id=match_id)
            G = digraph(passing_data_lyon)
            betweenness_centrality = avg(nx.betweenness_centrality(G))
            eigenvector_centrality = max(nx.eigenvector_centrality(G).values())
            katz_centrality = max(nx.katz_centrality(G).values())
            page_rank = max(nx.pagerank(G).values())
            
            
            match_data = {
                'Match ID': match_id,
                'Team': 'Lyon',
                'Betweenness Centrality': betweenness_centrality,
                'Eigenvector Centrality': eigenvector_centrality,
                'Katz Centrality': katz_centrality,
                'Page Rank': page_rank,
                'Average Clustering': (nx.average_clustering(G))
            }
            data = data.append(match_data, ignore_index=True)
        if (row['away_team_name'] == 'Lyon'):
            match_id = row['match_id']
            passing_data_lyon = pass_data_lyon(match_id=match_id)
            G = digraph(passing_data_lyon)
            betweenness_centrality = avg(nx.betweenness_centrality(G))
            eigenvector_centrality = max(nx.eigenvector_centrality(G).values())
            katz_centrality = max(nx.katz_centrality(G).values())
            page_rank = max(nx.pagerank(G).values())
            
            match_data = {
                'Match ID': match_id,
                'Team': 'Lyon',
                'Betweenness Centrality': betweenness_centrality,
                'Eigenvector Centrality': eigenvector_centrality,
                'Katz Centrality': katz_centrality,
                'Page Rank': page_rank,
                'Average Clustering': (nx.average_clustering(G))
            }
            data = data.append(match_data, ignore_index=True)

    return data

d = get_data(dat=teams)
d


NameError: name 'teams' is not defined

In [61]:
team_counts = d['Team'].value_counts()

print("Number of Lyon teams:", team_counts.get("Lyon", 0))
print("Number of AS Monaco teams:", team_counts.get("AS Monaco", 0))

Number of Lyon teams: 38
Number of AS Monaco teams: 38


In [62]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

In [139]:
X = d.drop(['Team','Match ID'], axis=1)
y = d['Team']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=33)
model = LogisticRegression()
model.fit(X_train, y_train)


LogisticRegression()

In [140]:
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
confusion = confusion_matrix(y_test, y_pred)
report = classification_report(y_test, y_pred)

In [141]:
print(f"Accuracy: {accuracy * 100:.2f}%")
print("Confusion Matrix:\n", confusion)
print("Classification Report:\n", report)

Accuracy: 42.11%
Confusion Matrix:
 [[ 1 10]
 [ 1  7]]
Classification Report:
               precision    recall  f1-score   support

   AS Monaco       0.50      0.09      0.15        11
        Lyon       0.41      0.88      0.56         8

    accuracy                           0.42        19
   macro avg       0.46      0.48      0.36        19
weighted avg       0.46      0.42      0.32        19



In [142]:
base_classifier = DecisionTreeClassifier(max_depth=6)  # You can adjust max_depth as needed

# Create the AdaBoost model
model = AdaBoostClassifier(base_classifier, n_estimators=20, random_state=33)

# Fit the model to the training data
model.fit(X_train, y_train)

# Make predictions on the test data
y_pred = model.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.5789473684210527
